# Results

## Library Import

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns

## Helper Functions

## Domain123 Baseline

In [ ]:
test_results = pd.read_csv(
    "../data/domain/results/domain123/2025-04-16_08-55-57/csv/version_0/test_results.csv"
)
test_results = test_results.map(
    lambda x: float(x.split("(")[-1].rstrip(")")) if isinstance(x, str) and "tensor" in x else x
)
test_results

In [ ]:
test_results_melted = test_results.melt(
    id_vars="sample_name",
    value_vars=["nmi", "homogeneity", "completeness"],
    var_name="Metric",
    value_name="Score",
)

sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(4, 2), sharex=True)

metrics = ["nmi", "homogeneity", "completeness"]
titles = ["NMI", "HOM", "COM"]

for ax, metric, title in zip(axes, metrics, titles):
    sns.boxplot(
        x="Score",
        y="Metric",
        data=test_results_melted[test_results_melted["Metric"] == metric],
        ax=ax,
        orient="h",
        dodge=False,
    )
    sns.stripplot(
        x="Score",
        y="Metric",
        data=test_results_melted[test_results_melted["Metric"] == metric],
        ax=ax,
        color="black",
        size=5,
        jitter=True,
        orient="h",
    )
    ax.set_xlim(0.2, 0.8)
    ax.set_title(title, fontsize=12)
    ax.set_xlabel("Score")
    ax.set_ylabel("")
    ax.set_yticklabels([])

    ax.set_xticks([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85])
    ax.set_xticklabels(["", "", "0.25", "", "", "", "", "0.75", ""])

plt.tight_layout()
plt.show()

In [ ]:
merfish_small5 = sc.read_h5ad(
    "../data/domain/results/domain123/2025-04-16_08-55-57/adata_files/MERFISH_small5.h5ad"
)
merfish_small5

In [ ]:
merfish_small5.obsm["spatial"] *= -1
merfish_small5.obsm["spatial"][:, 0] *= -1

fig, ax = plt.subplots(figsize=(5, 6))
sc.pl.embedding(
    merfish_small5,
    basis="spatial",
    color="domain_annotation",
    size=60,
    title="Ground Truth",
    ax=ax,
    show=False,
)
plt.show()

fig, ax = plt.subplots(figsize=(5, 6))
sc.pl.embedding(
    merfish_small5,
    basis="spatial",
    color="leiden",
    size=60,
    title="NMI: 0.68, HOM: 0.67, COM: 0.69",
    ax=ax,
    show=False,
)
plt.show()

In [ ]:
merfish_small5.uns["leiden_colors"] = [
    merfish_small5.uns["leiden_colors"][i] for i in [1, 0, 3, 4, 2, 7, 5, 6]
]